In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import verde as vd
import choclo
import harmonica as hm
import pandas as pd
import glob
import os
from scipy.interpolate import griddata
constant = choclo.constants.VACUUM_MAGNETIC_PERMEABILITY / (4 * np.pi)

from tetrahedron_forward import * 
import pathlib

In [ ]:
# save the xarray datasets into data folder

data_dir = pathlib.Path("/your/path")


In [ ]:
# Path to the folder where the files are located
folder_path = pathlib.Path("/your/path")

# List all files that end with "_packed.tec"
file_list = glob.glob(f"{folder_path}/*_packed.tec")

# Extract only the file names (without the full path)
file_names = [file.split("/")[-1] for file in file_list]

# Remove the suffix '_packed.tec' only from the variable
body_samples_names = sorted([file.replace("_packed.tec", "") for file in file_names])

# Display the results
print(body_samples_names)


In [ ]:
# Generating files in heights from a start to stop 
start = 0.1
stop = 10
num_points = 100  
heights = np.round(np.logspace(np.log10(start), np.log10(stop), num=num_points),2)
cube_top_um = 1.0          # Z coordinate of the cube’s top face (microns)


window_sizes = np.where(heights > 1.5, heights * 5, heights * 10)



for height, window_size in zip(heights, window_sizes):
    for body_sample in body_samples_names:
        dir_file_path = pathlib.Path("/your/path")

        # read the file data
        file_path =f'{body_sample}_mult.tec'
        file_path = dir_file_path / file_path
        file_content = read_tecplot_file(file_path)
        title, variables, N, E = read_tecplot_header(file_content)
        variables_values, sd_values, vertices_indexes = organize_data(file_content, N, E)
    
        # get the specific body index
        vertices_indexes_filtered = list()   
        for i, vertices_array in enumerate(vertices_indexes):
            if sd_values[i] == 101.0:
                vertices_indexes_filtered.append(np.asarray(vertices_array))
    
        # get nodes data
        X, Y, Z = variables_values['X'], variables_values['Y'], variables_values['Z']
        Mx, My, Mz = variables_values['Mx'], variables_values['My'], variables_values['Mz']
        # get tetrahedrons
        tetrahedrons, tetrahedrons_centroid = define_tetrahedrons(X, Y, Z, vertices_indexes_filtered)
        # sort tetrahedrons
        tetrahedrons_sorted = sort_tetrahedrons(tetrahedrons, tetrahedrons_centroid)
        # tetrahedrons magnetization
        Ms = 482476.91085597320
        tetrahedrons_mag = define_tetrahedrons_magnetizations_weighted(X, Y, Z, 
                                                                       Mx, My, Mz, 
                                                                       vertices_indexes_filtered, Ms=Ms)
        # get true magnetic moment of the grain
        true_magnetic_moment = calculate_grain_magnetic_moments(X, Y, Z, 
                                                                Mx, My, Mz, 
                                                                vertices_indexes_filtered, Ms=Ms)
            
        # generate coordinates grid
        c = np.mean([X, Y, Z], axis=1) # particles center
        x_windows_size = window_size   
        y_windows_size = window_size   
        spacing = (x_windows_size+y_windows_size)/100 # always make 101x101 grids
        
        
        sensor_sample_distance = np.max(Z) + height
        coordinates = vd.grid_coordinates(
            region=[-x_windows_size+c[0], x_windows_size+c[0], -y_windows_size+c[1], y_windows_size+c[1]],  # µm
            spacing=spacing,  # µm
            extra_coords=sensor_sample_distance,
        )
        
        coordinates = np.asarray(coordinates)*1.0e-6
        shape = np.shape(coordinates[0])
    
        # calculate model
        Bx_final = np.zeros(len(coordinates[0].ravel()))
        By_final = np.zeros(len(coordinates[0].ravel()))
        Bz_final = np.zeros(len(coordinates[0].ravel()))
        calculate_Omega_f_and_Delta_W_f(tetrahedrons_sorted, tetrahedrons_mag, coordinates, Bx_final, By_final, Bz_final)
    
        Bx_final = np.reshape(Bx_final, shape)
        By_final = np.reshape(By_final, shape)
        Bz_final = np.reshape(Bz_final, shape)
    
        plot = False
        if plot:
            plot_maps(Bx_final, By_final, Bz_final, coordinates, height, title=body_sample)
            print()
        # create the xarray dataset
        data = vd.make_xarray_grid(coordinates*1.0e6, Bz_final*1.0e9, data_names=["bz"], dims=("y", "x"), extra_coords_names="z")
        data.x.attrs = {"units": "µm"}
        data.y.attrs = {"units": "µm"}
        data.bz.attrs = {"long_name": "bz", "units": "nT"}

        # add the other components to the dataset if needed
        other_components = False
        if other_components:
            data = data.assign(bx=(['y','x'],Bx_final*1.0e9))
            data.bx.attrs = {"long_name": "bx", "units": "nT"}
            data = data.assign(by=(['y','x'],By_final*1.0e9))
            data.by.attrs = {"long_name": "by", "units": "nT"}
            b111 = np.array([0, np.sqrt(2/3), np.sqrt(1/3)])
            dot_product = Bx_final*b111[0]+By_final*b111[1]+Bz_final*b111[2]
            data = data.assign(b111=(['y','x'],dot_product*1.0e9))
            data.b111.attrs = {"long_name": "b111", "units": "nT"}
        
        # add the sensor height and the particle's centroid and magnetic moment as extra information 
        data = data.assign(sensor_height=height)
        data = data.assign(centroid=c)
        data = data.assign(magnetic_moment=true_magnetic_moment)
        data.centroid.attrs = {"long_name": "centroid position", "units": r"µm"}
        data.sensor_height.attrs = {"long_name": "sensor height", "units": r"µm"}
        data.magnetic_moment.attrs = {"long_name": "magnetic moment", "units": r"Am²"}

        data = data.assign(body_name=body_sample)
        
        # save the xarray into netcdf files
        data.to_netcdf(data_dir/f"grid_{body_sample}_{str(height)}.nc")
                    